# labelled sites example

In [1]:
from pymatgen.io.vasp import Poscar, Xdatcar
from pymatgen.symmetry.groups import SpaceGroup
from pymatgen import Lattice
import numpy as np
import operator
from site_analysis import Atom, Analysis, PolyhedralSite, get_vertex_indices, AtomsTrajectory, SitesTrajectory
from collections import Counter
import tqdm

Load a `POSCAR` file where every octahedral site is occupied by a Na atom.

In [2]:
all_na_structure = Poscar.from_file('na_sn_all_na_ext.POSCAR.vasp').structure
vertex_species = 'S'
centre_species = 'Na'

Create a series of pymatgen Structures using the `Structure.from_spacegroup()` method, that each only contain the NaX sites, using the coordinates from Ramos _et al._ _Chem. Mater._ 2018.

In [3]:
sg = SpaceGroup('I41/acd:2')

In [4]:
from pymatgen import Structure, Lattice
lattice = all_na_structure.lattice
na1 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.25, 0.0, 0.125]])
na2 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.00, 0.0, 0.125]])
na3 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.0, 0.25, 0.0]])
na4 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.0, 0.0, 0.0]])
na5 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.75, 0.25, 0.0]])
na6 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.5, 0.75, 0.625]])
i2  = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.666, 0.1376, 0.05]])
na_structures = {'Na1': na1,
                 'Na2': na2,
                 'Na3': na3,
                 'Na4': na4,
                 'Na5': na5,
                 'Na6': na6, 
                 'i2': i2}

In [5]:
s = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, 
                          species=['Sn','S','S','S', 'P','Na','Na','Na','Na','Na','K','Li','Mg'], 
                          coords=[[0.25, 0.0, 0.25],
                                  [0.326, 0.8997, 0.1987],
                                  [0.1258, 0.8972, 0.2999],
                                  [0.4097, 0.8339, 0.332],
                                  [0, 0.75, 0.125],
                                  [0.25, 0.0, 0.125],
                                  [0.0, 0.0, 0.125],
                                  [0.0, 0.25, 0.0],
                                  [0.0, 0.0, 0.0],
                                  [0.75, 0.25, 0.0],
                                  [0.5, 0.75, 0.625],
                                  [0.65, 0.375, 0.05],
                                  [0.25000,  0.75000,  0.00000]]) 

s.to(filename='all_atoms.cif')

Import the `matching_site_indices()` function from `polyhedral_analysis` (I should probably just add this to the `site-analysis` package). This function takes two pymatgen Structures as arguments, and finds the set of sites from the first that are closest to the sites in the second.

In [6]:
from polyhedral_analysis.polyhedra_recipe import matching_site_indices

In [7]:
print(matching_site_indices.__doc__)


    Returns a subset of site indices from structure (as a list) where each site is the closest to one 
    site in the reference structure.
    
    Args:
        structure (Structure): The structure being analysed.
        reference_structure (Structure): A Structure object containing a set of reference sites.
        species (:obj:`list[str]`, optional): A list of species labels. If this is set, only matching
            sites will be included in the returned set.
        
    Returns:
        (list[int])
    


In [8]:
matching_site_indices(all_na_structure, na_structures['Na6'])

[38, 37, 74, 73, 102, 103, 76, 75]

Use this to find the Na sites closest to a Na1, Na2, Na3 etc site, and store the site index and "Na1" etc. label in a dictionary, using the index as keys. Then sort this dictionary and generate an ordered list of "Na1" etc. labels for every Na site.

In [9]:
labels = {}
for l, structure in na_structures.items():
    indices = matching_site_indices(all_na_structure, structure)
    for i in indices:
        labels[i] = l
sorted_labels = [ labels[i] for i in sorted(labels) ]
print( sorted_labels )

['Na3', 'Na3', 'Na3', 'Na4', 'Na3', 'Na4', 'Na3', 'Na3', 'Na3', 'Na2', 'Na3', 'Na5', 'Na2', 'Na2', 'Na2', 'Na1', 'Na1', 'Na2', 'Na2', 'Na5', 'Na4', 'Na4', 'Na5', 'Na2', 'Na2', 'Na4', 'Na5', 'Na4', 'Na5', 'Na2', 'Na4', 'Na4', 'Na3', 'Na3', 'Na3', 'Na3', 'Na2', 'Na6', 'Na6', 'Na3', 'Na3', 'Na3', 'Na3', 'Na1', 'Na1', 'Na1', 'Na1', 'Na2', 'Na5', 'Na5', 'Na2', 'Na1', 'Na1', 'Na2', 'Na2', 'Na5', 'Na2', 'Na1', 'Na2', 'Na1', 'Na2', 'Na5', 'Na5', 'Na1', 'Na2', 'Na2', 'Na2', 'Na5', 'Na2', 'Na5', 'Na4', 'Na4', 'Na4', 'Na6', 'Na6', 'Na6', 'Na6', 'Na2', 'Na2', 'Na1', 'Na1', 'Na5', 'Na2', 'Na2', 'Na4', 'Na2', 'Na2', 'Na4', 'Na1', 'Na4', 'Na5', 'Na2', 'Na2', 'Na5', 'Na2', 'Na1', 'Na1', 'Na5', 'Na4', 'Na2', 'Na2', 'Na4', 'Na6', 'Na6', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2']


We then use the `get_vertex_indices()` function to find the six closest S to each Na (within a cutoff of 4.3 Å).  
This returns a nested list, where each sublist contains the S indices for a single polyedron.  
Note: this index counts from 1, and ignores other species in the structure (so is not affected by species order).

In [10]:
site_vertices = { 'Na1': 6,
                  'Na2': 6,
                  'Na3': 6,
                  'Na4': 6,
                  'Na5': 6,
                  'Na6': 8,
                  'i2': 6 }
n_vertices = [ site_vertices[l] for l in sorted_labels ]
print(n_vertices)

[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 8, 8, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 8, 8, 8, 8, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 8, 8, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]


In [11]:
# find atom indices (within species) for all polyhedra vertex atoms
vertex_indices = get_vertex_indices(all_na_structure, centre_species=centre_species, 
                                    vertex_species=vertex_species, cutoff=4.3, n_vertices=n_vertices)
print(vertex_indices[:4])

[[27, 29, 59, 61, 83, 85], [19, 21, 51, 53, 91, 93], [9, 15, 41, 47, 65, 71], [7, 23, 33, 49, 73, 90]]


We can now use these vertex ids to define our `Polyhedron` objects.   
We now also pass in the appropriate label to each polyhedron.

In [12]:
structure = Poscar.from_file('POSCAR').structure
# create Polyhedron objects
sites = [PolyhedralSite(vertex_species=vertex_species, vertex_indices=vi, label=label) 
             for vi, label in zip(vertex_indices, sorted_labels)]
# create Atom objects
atoms = [Atom(species_string=centre_species) for site in structure if site.species_string is 'Na']
analysis = Analysis(sites, atoms)

In [13]:
analysis.site_coordination_numbers()

Counter({6: 128, 8: 8})

Polyhedra labels can be accessed directly, or as a list:

In [14]:
analysis.sites[0].label

'Na3'

In [15]:
print(analysis.site_labels())

['Na3', 'Na3', 'Na3', 'Na4', 'Na3', 'Na4', 'Na3', 'Na3', 'Na3', 'Na2', 'Na3', 'Na5', 'Na2', 'Na2', 'Na2', 'Na1', 'Na1', 'Na2', 'Na2', 'Na5', 'Na4', 'Na4', 'Na5', 'Na2', 'Na2', 'Na4', 'Na5', 'Na4', 'Na5', 'Na2', 'Na4', 'Na4', 'Na3', 'Na3', 'Na3', 'Na3', 'Na2', 'Na6', 'Na6', 'Na3', 'Na3', 'Na3', 'Na3', 'Na1', 'Na1', 'Na1', 'Na1', 'Na2', 'Na5', 'Na5', 'Na2', 'Na1', 'Na1', 'Na2', 'Na2', 'Na5', 'Na2', 'Na1', 'Na2', 'Na1', 'Na2', 'Na5', 'Na5', 'Na1', 'Na2', 'Na2', 'Na2', 'Na5', 'Na2', 'Na5', 'Na4', 'Na4', 'Na4', 'Na6', 'Na6', 'Na6', 'Na6', 'Na2', 'Na2', 'Na1', 'Na1', 'Na5', 'Na2', 'Na2', 'Na4', 'Na2', 'Na2', 'Na4', 'Na1', 'Na4', 'Na5', 'Na2', 'Na2', 'Na5', 'Na2', 'Na1', 'Na1', 'Na5', 'Na4', 'Na2', 'Na2', 'Na4', 'Na6', 'Na6', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2']


In [16]:
analysis.append_timestep(structure)

The occupations of each site are stored as a list of lists, as each site can have zero, one, or multiple atoms occupying it.

In [17]:
print(analysis.st[0])

[[51], [62], [65], [35], [33], [31], [47], [39], [57], [1], [], [41], [18], [20], [4], [16], [5], [76], [87], [], [72], [67], [53], [8], [26], [70], [45], [], [], [], [58], [61], [29], [37], [30], [36], [74], [], [], [40], [32], [38], [34], [84], [80], [85], [73], [], [50], [43], [22], [77], [82], [25], [14], [56], [12], [83], [11], [81], [23], [55], [48], [78], [15], [17], [7], [44], [6], [49], [68], [63], [60], [], [], [], [], [], [27], [88], [75], [46], [28], [10], [71], [24], [13], [69], [79], [66], [54], [19], [2], [42], [21], [], [9], [52], [64], [3], [86], [59], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


Rough example for collecting only occupied sites, and counting their site types:

In [18]:
c = Counter()
for site in analysis.sites:
    c[site.label] += len([ 1 for ts in site.trajectory if len(ts)>0 ])
c

Counter({'Na3': 15,
         'Na4': 15,
         'Na2': 29,
         'Na5': 14,
         'Na1': 15,
         'Na6': 0,
         'i2': 0})

vs. all sites:

In [19]:
c_sites = Counter(analysis.site_labels())
c_sites

Counter({'Na3': 16,
         'Na4': 16,
         'Na2': 32,
         'Na5': 16,
         'Na1': 16,
         'Na6': 8,
         'i2': 32})

In [20]:
analysis.reset()

xdatcar = Xdatcar('XDATCAR_Sn')

analysis.trajectory_from_structures( xdatcar.structures, progress='notebook')

In [21]:
n_timesteps = len(analysis.timesteps)
c_sites = Counter(analysis.site_labels())
c = Counter()
p_occ = {}
for site in analysis.sites:
    c[site.label] += len([ 1 for ts in site.trajectory if len(ts)>0 ])
for k, v in c.items():
    p_occ[k] = v / c_sites[k] / n_timesteps
p_occ

{'Na3': 0.911875,
 'Na4': 0.945625,
 'Na2': 0.8944791666666667,
 'Na5': 0.915,
 'Na1': 0.9354166666666667,
 'Na6': 0.0,
 'i2': 0.0015625}

In [22]:
# check total average occupation = 88 atoms
for k,v in c.items():
    print( k, p_occ[k]*c_sites[k])
print( sum( [ p_occ[k] * c_sites[k] for k, v in c.items()]))

Na3 14.59
Na4 15.13
Na2 28.623333333333335
Na5 14.64
Na1 14.966666666666667
Na6 0.0
i2 0.05
88.0
